In [ ]:
import pandas as pd
import numpy as np
import sqlite3 as db
from datetime import datetime
import urllib
import os



con = db.connect(database="../db.sqlite")

In [ ]:

cursor = con.cursor()
cursor.execute("select CountryCode, Name from Country")

list = []

for row in cursor.fetchall():
    list.append({
        "CountryCode": row[0],
        "Country": row[1]
    })
    

def getCountryCode(name):
    
    if name == "United Kingdom":
        return "GBR"
    elif name == "Russian Federation":
        return "RUS"
    elif name == "Korea":
        return "KOR"
    elif name == "Slovak Republic":
        return "SVK"
    
    for c in list:
        if c["Country"] == name:
            return c["CountryCode"]
        
    return None

In [ ]:
xl = pd.ExcelFile("Average Tax Wedges 2020.xlsx")

df = xl.parse(0)

df["Source"] = "oecd"
df["Country"] = df["Country"].apply(lambda x: x.strip())

In [ ]:
## replace with country code

dfKeys = df.groupby("Country").groups.keys()


for key in dfKeys:
    if getCountryCode(key) == None:
        print(key + " does not exist in db")
        
        
df["Country"] = df["Country"].apply(lambda x: getCountryCode(x))

In [ ]:
dfFam2 = df[["Country", "Family 2 Total Tax Wedge", "Source"]]
dfFam2.rename(columns={"Family 2 Total Tax Wedge": "Rate", "Country": "Country_id"},inplace=True)
dfFam2["Rate"] = dfFam2["Rate"] / 100
dfFam2["Profession"] = "General Family of 2"
dfFam2["Date"] = datetime(2020,1,1)
dfFam2.to_sql("TaxWedge", con,if_exists="append", index=False)

In [ ]:
dfSingle = df[["Country", "Single Tax Wedge", "Source"]]
dfSingle.rename(columns={"Single Tax Wedge": "Rate", "Country": "Country_id"},inplace=True)
dfSingle["Rate"] = dfSingle["Rate"] / 100
dfSingle["Profession"] = "General"
dfSingle["Date"] = datetime(2020,1,1)
dfSingle.to_sql("TaxWedge", con,if_exists="append", index=False)

In [ ]:
df["Family 2 Total Tax Wedge"] = df["Family 2 Total Tax Wedge"] - df["Family Annual Change"]
df["Single Tax Wedge"] = df["Single Tax Wedge"] - df["Single Annual Change"]

In [ ]:
dfFam2 = df[["Country", "Family 2 Total Tax Wedge", "Source"]]
dfFam2.rename(columns={"Family 2 Total Tax Wedge": "Rate", "Country": "Country_id"},inplace=True)
dfFam2["Rate"] = dfFam2["Rate"] / 100
dfFam2["Profession"] = "General Family of 2"
dfFam2["Date"] = datetime(2019,1,1)
dfFam2.to_sql("TaxWedge", con,if_exists="append", index=False)

In [ ]:
dfSingle = df[["Country", "Single Tax Wedge", "Source"]]
dfSingle.rename(columns={"Single Tax Wedge": "Rate", "Country": "Country_id"},inplace=True)
dfSingle["Rate"] = dfSingle["Rate"] / 100
dfSingle["Profession"] = "General"
dfSingle["Date"] = datetime(2019,1,1)
dfSingle.to_sql("TaxWedge", con,if_exists="append", index=False)